In [1]:
import csv
import re
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz
import pafy

In [2]:
def deEmojify(text):
    regrex_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)
    return regrex_pattern.sub(r'',text)

In [3]:
api_link = 'https://www.googleapis.com/youtube/v3/playlistItems?part=snippet&maxResults=50&playlistId={channel_id}&key={api_key}&pageToken={next_token}'
api_key = APIKEY
channel_links = {'TrapNation': 'https://www.youtube.com/c/TrapNation',
                 'TrapCity': 'https://www.youtube.com/user/OfficialTrapCity',
                 'BassNation': 'https://www.youtube.com/channel/UCCvVpbYRgYjMN7mG7qQN0Pg',
                 'UKFDubstep': 'https://www.youtube.com/user/UKFDubstep',
                 'BassMusicMovement': 'https://www.youtube.com/c/BassMusicMovement',
                 'DubRebellion': 'https://www.youtube.com/user/RebellionOfficial'
                }


In [4]:
# Access all videos of a channel through YT API and store relevant data line-by-line in .csv

with open('allvideos.csv', 'w', encoding='utf-16', newline='') as csv_file:
    fieldnames = ['channelName', 'channelID', 'title', 'videoID', 'datePublished']

    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    
    for channel in channel_links:
        channel_page = requests.get(channel_links[channel])
        soup = BeautifulSoup(channel_page.content, 'html.parser')
        channel_id = list(soup.find('meta', itemprop="channelId")['content'])
        channel_id[1] = 'U'
        channel_id = "".join(channel_id)
        
        next_token = ''
        end = False
        # loop through all pages
        while not end:
            temp = {}
            channel_api = api_link.format(channel_id = channel_id, api_key = api_key, next_token = next_token)
            channel_data = requests.get(channel_api).json()
            try:
                next_token = channel_data['nextPageToken']
            except KeyError:
                end = True
            
            for i in range(len(channel_data['items'])):
                temp['channelName'] = channel
                temp['channelID'] = channel_id
                temp['title'] = deEmojify(channel_data['items'][i]['snippet']['title'])
                temp['datePublished'] = channel_data['items'][i]['snippet']['publishedAt'][:10]
                temp['videoID'] = channel_data['items'][i]['snippet']['resourceId']['videoId']
                
                writer.writerow(temp)
        

In [5]:
df = pd.read_csv('allvideos.csv', encoding='utf16')
df.shape

(10655, 5)

In [6]:
# clean up video titles

df['title'] =  df['title'].apply(lambda x: x.replace('[','(').replace(']',')')) 
remove = re.compile(r' \((Official|OFFICIAL)?(( )?(Music|Lyric)? Video)?\)|(Lyrics)')
df['title'] = [remove.sub('', x) for x in df['title']]
remove2 = re.compile(r' \(Royalty Free( Music)?\)')
df['title'] = [remove2.sub('', x) for x in df['title']]

In [7]:
# split title by "-" into trackname and artist

df[['artists','track']] = df['title'].str.split(' - ', 1, expand=True)
df.head()

,channelName,channelID,title,videoID,datePublished,artists,track
0,TrapNation,UUa10nxShhzNrCE1o2ZOPztg,TroyBoi - Mantra,YN9R8IRXv5g,2021-01-25,TroyBoi,Mantra
1,TrapNation,UUa10nxShhzNrCE1o2ZOPztg,Juche - NFS,FelgrmPU-FE,2021-01-23,Juche,NFS
2,TrapNation,UUa10nxShhzNrCE1o2ZOPztg,Skeler - Akita,A29u2rOFlB4,2021-01-21,Skeler,Akita
3,TrapNation,UUa10nxShhzNrCE1o2ZOPztg,Baauer - Aether (Juelz & Heimanu Remix),05DF303eFso,2021-01-19,Baauer,Aether (Juelz & Heimanu Remix)
4,TrapNation,UUa10nxShhzNrCE1o2ZOPztg,Axel Thesleff - Unity,XpEuJY_mgVI,2021-01-17,Axel Thesleff,Unity


In [8]:
# drop rows with titles that were not in the form of Artist - Track

df['track'].replace('', np.nan, inplace=True)
df.dropna(subset=['track'], inplace=True)
df.shape

(10284, 7)

In [9]:
collabs = ' & ', ' + ', ' x ', 'vs\. '
collabs_pattern = '|'.join(collabs) # regex to look for for multiple artists 
df['artists'] = [re.sub(collabs_pattern, ', ', x.lower()) for x in df['artists']] #sub collab tags with comma for cleaner Spotify search

In [10]:
features = '( )?(ft(\.)? )( )?', '( )?(feat(\.)? )( )?',  '( )?(w/)( )?', '( )?(Ft(\.)? )( )?', '( )?(Feat(\.)? )( )?'
features_pattern = '|'.join(features)
df['artists'] = [re.sub(features_pattern, ', ', x) for x in df['artists']] #sub feature tags with comma 
df['track'] = [re.sub(features_pattern, 'feat. ', x) for x in df['track']] #sub feature tags with consistent 'feat. ' 

In [11]:
# clean up trackname to ease searching on Spotify

df[['strippedTrackName']] = [re.sub(r'[^\w]', ' ', x) for x in df['track']]
df['strippedTrackName'] = [re.sub(features_pattern, ' ', x) for x in df['strippedTrackName']]

In [12]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials 
import json
import time

client_id = ID
client_secret = APIKEY
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) #spotify object to access API

In [13]:
audio_features = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 
                  'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 
                  'duration_ms', 'time_signature']

df_ytsp = df.copy()
df_ytsp['yt_duration'] = ''
df_ytsp['yt_views'] = ''
df_ytsp['s_id'] = ''
df_ytsp['s_track'] = ''
df_ytsp['s_artists'] = ''
df_ytsp['s_release_date'] = ''
df_ytsp['popularity'] = ''


for f in audio_features:
    df_ytsp[f] = ''

df_ytsp.head()

,channelName,channelID,title,videoID,datePublished,artists,track,strippedTrackName,yt_duration,yt_views,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,TrapNation,UUa10nxShhzNrCE1o2ZOPztg,TroyBoi - Mantra,YN9R8IRXv5g,2021-01-25,troyboi,Mantra,Mantra,,,...,,,,,,,,,,
1,TrapNation,UUa10nxShhzNrCE1o2ZOPztg,Juche - NFS,FelgrmPU-FE,2021-01-23,juche,NFS,NFS,,,...,,,,,,,,,,
2,TrapNation,UUa10nxShhzNrCE1o2ZOPztg,Skeler - Akita,A29u2rOFlB4,2021-01-21,skeler,Akita,Akita,,,...,,,,,,,,,,
3,TrapNation,UUa10nxShhzNrCE1o2ZOPztg,Baauer - Aether (Juelz & Heimanu Remix),05DF303eFso,2021-01-19,baauer,Aether (Juelz & Heimanu Remix),Aether Juelz Heimanu Remix,,,...,,,,,,,,,,
4,TrapNation,UUa10nxShhzNrCE1o2ZOPztg,Axel Thesleff - Unity,XpEuJY_mgVI,2021-01-17,axel thesleff,Unity,Unity,,,...,,,,,,,,,,


In [18]:
def searchSpotify(dataframe):
    sleep_min = 5
    sleep_max = 10
    start_time = time.time()
    request_count = 0
    url = 'http://www.youtube.com/watch?v={id}'

    for index, row in dataframe.iterrows():
        if request_count % 5 == 0:
            print(str(request_count) + " tracks searched")
            time.sleep(np.random.uniform(sleep_min, sleep_max))
            print('Elapsed Time: {} seconds'.format(time.time() - start_time))    
        
        # using Pafy library to query YT video length and count to avoid hitting API call limit
        try:
            dataframe.at[index, 'yt_duration'] = pafy.new(url.format(id=str(row['videoID']))).length
            dataframe.at[index, 'yt_views'] = pafy.new(url.format(id=str(row['videoID']))).viewcount
        except:
            dataframe.at[index, 'yt_duration'] = -1
            dataframe.at[index, 'yt_views'] = -1

        query = row['artists'] + ' ' + row['strippedTrackName']
        
        # search track with Spotify API
        result = sp.search(query.lower(), limit=5, market='US')
        num_of_results = len(result['tracks']['items'])
        if num_of_results == 0:
            print('not found: ' + query)
        else:
            max_ratio = 0
            item_index = 0
            criteria_met = False
            
            # FuzzyWuzzy Levenshtein Distance string comparison method. 
            # Get Spotify data if matched tracknames are > 50 match ratio
            for item in range(num_of_results):
                ratio = fuzz.ratio(result['tracks']['items'][item]['name'], row['track'])
                if ratio > max_ratio:
                    max_ratio = ratio
                    item_index = item

            if max_ratio > 50:
                criteria_met = True
            
            # Compare track durations as a backup method
            else:
                for item in range(num_of_results):
                    duration = round(float(result['tracks']['items'][item]['duration_ms'])/1000, 1)
                    if float(row['yt_duration'])-0.6 < duration < float(row['yt_duration'])+0.6:
                        item_index = item
                        criteria_met = True
                        break

            if criteria_met:
                s_id = result['tracks']['items'][item_index]['id']
                dataframe.at[index, 's_id'] = s_id
                dataframe.at[index, 's_track'] = result['tracks']['items'][item_index]['name']
                dataframe.at[index, 's_release_date'] = result['tracks']['items'][item_index]['album']['release_date']
                dataframe.at[index, 'popularity'] = result['tracks']['items'][item_index]['popularity']
                spotify_artists = []
                for name in range(len(result['tracks']['items'][item_index]['artists'])):
                    spotify_artists.append(result['tracks']['items'][item_index]['artists'][name]['name'])
                dataframe.at[index, 's_artists'] = spotify_artists

                # Get audio features data of Spotify ID
                features = sp.audio_features(s_id)
                for f in audio_features:
                    try:
                        dataframe.at[index, f] = features[0][f]
                    except TypeError:
                        dataframe.at[index, f] = None
            else:
                pass

        request_count+=1

In [15]:
def findPercentages(dataframe):
    print('# of Tracks Found on SP: ' + str(len(dataframe[dataframe['s_id'] != ''])) + 
          ' (Percentage: ' + str(len(dataframe[dataframe['s_id'] != '']) / len(dataframe)) + ')')

    print('TrapNation: ' + str(len(dataframe[(dataframe['s_id'] != '') & (dataframe['channelName'] == 'TrapNation')])) + 
          ' (Percentage: ' + str(len(dataframe[(dataframe['s_id'] != '') & (dataframe['channelName'] == 'TrapNation')]) 
                           / len(dataframe[dataframe['channelName'] == 'TrapNation'])) + ')')

    print('TrapCity: ' + str(len(dataframe[(dataframe['s_id'] != '') & (dataframe['channelName'] == 'TrapCity')])) + 
          ' (Percentage: ' + str(len(dataframe[(dataframe['s_id'] != '') & (dataframe['channelName'] == 'TrapCity')]) 
                           / len(dataframe[dataframe['channelName'] == 'TrapCity'])) + ')')

    print('BassNation: ' + str(len(dataframe[(dataframe['s_id'] != '') & (dataframe['channelName'] == 'BassNation')])) + 
          ' (Percentage: ' + str(len(dataframe[(dataframe['s_id'] != '') & (dataframe['channelName'] == 'BassNation')]) 
                           / len(dataframe[dataframe['channelName'] == 'BassNation'])) + ')')

    print('UKFDubstep: ' + str(len(dataframe[(dataframe['s_id'] != '') & (dataframe['channelName'] == 'UKFDubstep')])) + 
          ' (Percentage: ' + str(len(dataframe[(dataframe['s_id'] != '') & (dataframe['channelName'] == 'UKFDubstep')]) 
                           / len(dataframe[dataframe['channelName'] == 'UKFDubstep'])) + ')')

In [19]:
searchSpotify(df_ytsp)

0 tracks searched
Elapsed Time: 8.139764547348022 seconds
not found: baauer Aether  Juelz   Heimanu Remix 
5 tracks searched
Elapsed Time: 25.42373251914978 seconds
10 tracks searched
Elapsed Time: 40.689292907714844 seconds
15 tracks searched
Elapsed Time: 58.294432640075684 seconds
20 tracks searched
Elapsed Time: 73.62937259674072 seconds
not found: wham! Last Christmas  San Holo Remix 
not found: baauer Aether  Skeler Remix 
25 tracks searched
Elapsed Time: 89.01807522773743 seconds
not found: danileigh No Limits  Skeler Remix 
30 tracks searched
Elapsed Time: 105.00038409233093 seconds
not found: post malone Circles  Haywyre Flip 
35 tracks searched
Elapsed Time: 121.20086979866028 seconds
40 tracks searched
Elapsed Time: 135.04406785964966 seconds
not found: camoufly everybody x2
not found: beyonce, destiny's child JUMP BROS   LYNY Remix 
not found: l*o*j Medusa In Naboo
not found: pitbull Hotel Room Service  camoufly Remix 
45 tracks searched
Elapsed Time: 149.36097288131714 sec

not found: kanye west Closed On Sunday  Elijah Hill Remix 
not found: kanye west Use This Gospel  Biicla Remix Clipse  Kenny G
360 tracks searched
Elapsed Time: 1116.6253490447998 seconds
not found: nirvana Heart Shaped Box  DIVMOND   Cafe Disko Cover 
not found: tame impala The Less I Know The Better  SAINT WKND Flip 
365 tracks searched
Elapsed Time: 1133.7267377376556 seconds
not found: kanye west Heartless  Miles Away   Braev Remix 
not found: shawn mendes, camila cabello Señorita  Hopex Remix 
not found: louis the child Better Not Wafia  INZO Remix 
370 tracks searched
Elapsed Time: 1146.4046125411987 seconds
375 tracks searched
Elapsed Time: 1161.1701412200928 seconds
not found: lil tecca Ransom  Naderi Remix 
not found: maroon 5 Memories  Elijah Hill Remix 
not found: stranger things Friends Don t Lie  Haterade Remix 
380 tracks searched
Elapsed Time: 1178.998922586441 seconds
385 tracks searched
Elapsed Time: 1197.0214111804962 seconds
not found: 5 seconds of summer Teeth  Tim 

Elapsed Time: 1900.1738736629486 seconds
not found: stephen Play Me Like A Violin  Meisym x Flayx Remix 
not found: odesza Just A Memory Regina Spektor  PURGE Remix 


ERROR: This video contains content from WMG and [Merlin] Ignition, one or more of whom have blocked it on copyright grounds.


not found: oasis Wonderwall  Arcando   Lux Holm Remix 
not found: g eazy, halsey Him   I  Vanic Remix 
625 tracks searched
Elapsed Time: 1911.6622989177704 seconds
not found: g eazy Eazy Son Lux  Vanic Remix 
not found: ariana grande thank u  next  Neovaii Remix 
630 tracks searched
Elapsed Time: 1928.3271493911743 seconds
not found: sasha sloan The Only  TWO LANES Remix 
not found: droeloe Step By Step Iris Penning  Fytch Remix 
not found: mstrmnd Battlefield
635 tracks searched
Elapsed Time: 1943.2557864189148 seconds
640 tracks searched
Elapsed Time: 1956.3811588287354 seconds
not found: 6ix9ine KIKA Tory Lanez  BEAUZ Remix 
645 tracks searched
Elapsed Time: 1972.3314425945282 seconds
not found: post malone, swae lee Sunflower  Not Your Dope Remix 
not found: jauz, dj snake Gassed Up  Hyperclap Remix 
not found: sheck wes Mo Bamba  Jaydon Lewis x Afterfab Remix 
not found: halsey Strangers Lauren Jauregui  Nolan van Lith Remix 
650 tracks searched
Elapsed Time: 1985.6965081691742 se

...

not found: styn, obey Pyrex
not found: mr. carmack BEGGIN  STRIPS
9745 tracks searched
Elapsed Time: 29044.927780866623 seconds
not found: half empty Tingles Magic Words
not found: m a z e Businessman
not found: ad, subfiltronik, curzed Evil Ryu  AR the Bushmaster Remix 
9750 tracks searched
Elapsed Time: 29060.779614925385 seconds
not found: 50 carrot Nark
not found: skream Midnight Request Line  D2 Remix 
9755 tracks searched
Elapsed Time: 29073.475214004517 seconds
not found: hizzleguy Drunk Dub VIP
not found: deadexit Snake
not found: cheshire Good Time  AR the Bushmaster Remix 
9760 tracks searched
Elapsed Time: 29089.140750408173 seconds
not found: skream, plastician We Like The Dark
not found: dubloadz Ill Communication
not found: 12gauge Clappa Hunde
not found: karnage, markiv Zenith
9765 tracks searched
Elapsed Time: 29105.513688325882 seconds
not found: squnto Sirens
not found: trollphace Heretic
9770 tracks searched
Elapsed Time: 29122.056470155716 seconds
not found: barely 

ERROR: This video contains content from SME, who has blocked it in your country on copyright grounds.


not found: dj snake BBC Radio 1 Essential Mix  1 25 14 
9870 tracks searched
Elapsed Time: 29431.30950474739 seconds
not found: (drum&bass) nelver Simple Miracle
not found: (dubstep) deadexit My Life In Your Hands
not found: (dubstep) rise at night Another One  Dynamic MC   SPL Remix 
not found: (dubstep) genetix, sadhu Sound The Alarm
not found: (dubstep) xaebor, exude Dingus Flex
9875 tracks searched
Elapsed Time: 29446.25276207924 seconds
not found: (dubstep) wicked fat noize Big Vein
not found: (dubstep) ponicz The Record Shop
not found: (dubstep) grimblee Bully
not found: dj snake Australia New Zealand 2013 Tour  Official Aftermovie 
not found: (dubstep) moth 666
9880 tracks searched
Elapsed Time: 29465.32337450981 seconds
not found: (trap) dj snake vs prodigy, mercer Breathe  Parisian Vision   Official Audio 
not found: (dubstep) alexisonfire To A Friend  Twofold Remix 
not found: (trap) baauer Snap  HugLife Remix 
not found: (dubstep) woogie Military
not found: (dubstep) eptic G

not found: (dubstep) allesnik Freeway Swervin 
not found: (trap) mr carmack, arnold Muney  Gimme That   LOUDPVCK Edit 
not found: (dubstep) neosignal Planet Online  Culprate Remix 
10005 tracks searched
Elapsed Time: 29875.990474700928 seconds
not found: (drumstep) datsik Release Me  Barron Remix 
not found: (dubstep) the juggernaut Sock Hop
not found: (dubstep) subfiltronik Passout  Skullion Shadez 
not found: (drum&bass) carvar, clock Miskatonik  Joe Ford Remix 
not found: (dubstep) megalodon Bad Mind
10010 tracks searched
Elapsed Time: 29892.900964021683 seconds
not found: (trap) funtcase Out For Da Milli
not found: (dubstep) d-jahsta, 12gauge, sadhu Baptism Of Fire  Phiso VIP 
not found: (trap) 12th planet, mayhem Murdaaa
not found: (drum&bass) dez Dat Riddim
not found: (dubstep) datsik Complete Control  The Frim Remix 
10015 tracks searched
Elapsed Time: 29908.96388220787 seconds
not found: (dubstep) juju, trollphace Imagination
not found: (dubstep) soulja boy Turn My Swag On  Jan

not found: (dubstep) funtcase So Vexed  Cookie Monsta Remix 
not found: (dubstep) exude Decapitate
not found: (dubstep) subshock G A R
10135 tracks searched
Elapsed Time: 30299.316719055176 seconds
not found: (dubstep) rekoil Astro Boy
not found: (dubstep) k12 Show Me Messy
not found: (drum&bass) heist Unicron VIP
not found: (dubstep) jphelpz Sentry Gun  Exude Remix 
not found: (dubstep) fuzion Midnight Ride  Sunn   Rameses B Remix 
10140 tracks searched
Elapsed Time: 30316.979231357574 seconds
not found: (dubstep) ponicz Ghetto Blaster
not found: (dubstep) sadhu Say What  Exude Remix 
not found: (drum&bass) seven lions Days To Come  Dead Battery Remix 
not found: (dubstep) northern lights Sentiments Beezy
not found: (dubstep) tom encore Hyperfunk
10145 tracks searched
Elapsed Time: 30332.033804893494 seconds
not found: (dubstep) klrgrm, suspect Nuff Said
not found: (trap) ruddyp Maybe
not found: (dubstep) mark instinct, charlotte d Silence
10150 tracks searched
Elapsed Time: 30351.297

In [ ]:
findPercentages(df_ytsp)

In [20]:
df_ytsp.to_csv('combined_data.csv')

In [17]:
df.to_csv('youtube_data.csv')